In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

In [2]:
spark = (
    SparkSession.builder
    .appName("sdmf")
    .enableHiveSupport()
    .config(
        "spark.jars.packages",
        "io.delta:delta-spark_2.12:3.1.0"
    )
    .config(
        "spark.sql.extensions",
        "io.delta.sql.DeltaSparkSessionExtension"
    )
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    )
    .getOrCreate()
)


your 131072x1 screen size is bogus. expect trouble
26/01/28 10:38:37 WARN Utils: Your hostname, DESKTOP-8BFCOG4 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/28 10:38:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hando/dev/standard-data-management-framework/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hando/.ivy2/cache
The jars for the packages stored in: /home/hando/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-32d5018f-9363-4b4a-83e2-5402a3442c5a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 103ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

In [12]:
df = spark.range(1, 101).toDF("row_id")
df = (
    df
    .withColumn("alpha3_b", expr("concat('USA', row_id)"))
    .withColumn("alpha3_t", expr("concat('US', row_id)"))
    .withColumn("alpha2", expr("substring('US', 1, 2)"))
    .withColumn(
        "english",
        expr("""
            CASE
                WHEN row_id % 4 = 0 THEN 'United States'
                WHEN row_id % 4 = 1 THEN 'Germany'
                WHEN row_id % 4 = 2 THEN 'India'
                ELSE 'Canada'
            END
        """)
    )
    .drop("row_id")
)
spark.sql("CREATE DATABASE IF NOT EXISTS demo")
spark.sql("DROP TABLE IF EXISTS demo.customers")
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("demo.customers")

26/01/24 17:46:55 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`demo`.`customers` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


In [13]:
spark.catalog.tableExists("demo.customers")

True

In [14]:

import random
import string

# Function to generate random values
def random_alpha3():
    return ''.join(random.choices(string.ascii_uppercase, k=3))

def random_alpha2():
    return ''.join(random.choices(string.ascii_uppercase, k=2))

def random_english_word():
    return ''.join(random.choices(string.ascii_lowercase, k=6))

# Generate random values
alpha3_b = random_alpha3()
alpha3_t = random_alpha3()
alpha2 = random_alpha2()
english = random_english_word()

# Insert into Spark SQL
query = f"""
INSERT INTO demo.customers (alpha3_b, alpha3_t, alpha2, english)
VALUES ('{alpha3_b}', '{alpha3_t}', '{alpha2}', '{english}')
"""

spark.sql(query)


DataFrame[]

In [19]:
spark.sql("select * from demo.customers").show(10000000, truncate=False)

+--------+--------+------+-------------+
|alpha3_b|alpha3_t|alpha2|english      |
+--------+--------+------+-------------+
|USA96   |US96    |US    |United States|
|USA97   |US97    |US    |Germany      |
|USA98   |US98    |US    |India        |
|USA99   |US99    |US    |Canada       |
|USA100  |US100   |US    |United States|
|USA91   |US91    |US    |Canada       |
|USA92   |US92    |US    |United States|
|USA93   |US93    |US    |Germany      |
|USA94   |US94    |US    |India        |
|USA95   |US95    |US    |Canada       |
|USA41   |US41    |US    |Germany      |
|USA42   |US42    |US    |India        |
|USA43   |US43    |US    |Canada       |
|USA44   |US44    |US    |United States|
|USA45   |US45    |US    |Germany      |
|USA26   |US26    |US    |India        |
|USA27   |US27    |US    |Canada       |
|USA28   |US28    |US    |United States|
|USA29   |US29    |US    |Germany      |
|USA30   |US30    |US    |India        |
|USA21   |US21    |US    |Germany      |
|USA22   |US22  